In [0]:
# Running PySpark at Google Colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
data = spark.read.csv('/content/drive/My Drive/Colab Notebooks/original.csv', inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [14]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [10]:
for item in data.head(1)[0]:
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [17]:
for item in data.head(2)[1]:
  print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [19]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'], outputCol='features')

In [0]:
output = assembler.transform(data)

In [22]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [23]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [0]:
final_data = output.select('features', 'Yearly Amount Spent')

In [25]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [0]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [42]:
train_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[29.5324289670579...|  408.6403510726275|
|[30.3931845423455...|  319.9288698031936|
|[30.4925366965402...|  282.4712457199145|
|[30.5743636841713...| 442.06441375806565|
|[30.8364326747734...|  467.5019004269896|
|[30.8794843441274...|  490.2065999848547|
|[30.9716756438877...|  494.6386097568927|
|[31.0472221394875...|  392.4973991890214|
|[31.0613251567161...|  487.5554580579016|
|[31.0662181616375...| 448.93329320767435|
|[31.1239743499119...|  486.9470538397658|
|[31.1280900496166...|  557.2526867470547|
|[31.1695067987115...|  427.3565308022928|
|[31.2606468698795...|  421.3266312569514|
|[31.2681042107507...|  423.4705331738239|
|[31.3091926408918...|  432.7207178399336|
|[31.3584771924370...|  495.1759504494754|
|[31.3662121671876...|  430.5888825564849|
|[31.3895854806643...|  410.0696110599829|
|[31.4252268808548...|  530.7667186547619|
+----------

In [28]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                342|
|   mean| 497.80784387313435|
| stddev|  80.70452178437795|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [29]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                158|
|   mean|  502.5742818017935|
| stddev|   76.3691673819774|
|    min|   266.086340948469|
|    max|  725.5848140556806|
+-------+-------------------+



In [0]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [0]:
lr_model = lr.fit(train_data)

In [0]:
test_results = lr_model.evaluate(test_data)

In [33]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 11.924464117649336|
|-16.121845886994265|
|  23.50809307323334|
| 19.797316419408446|
| -4.263757761287366|
| 3.6237990823320274|
|-16.832310678415297|
|-0.9813286430738799|
| 17.729782375000525|
| -4.496838618325285|
|  6.980754816513979|
|  -9.77289313176766|
|-17.699889329670043|
|-0.8388541126057021|
| 1.9019967873241512|
|-1.7213297532475735|
|-10.267896916314271|
|-16.401562088938192|
|-13.044254469360794|
| -4.638785277948614|
+-------------------+
only showing top 20 rows



In [34]:
test_results.rootMeanSquaredError

10.672555185994657

In [35]:
test_results.r2

0.9803456746277767

In [37]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
unlabeled_data = test_data.select('features')

In [39]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.7377203726281...|
|[30.8162006488763...|
|[31.2834474760581...|
|[31.3123495994443...|
|[31.5257524169682...|
|[31.5316044825729...|
|[31.5702008293202...|
|[31.5761319713222...|
|[31.6098395733896...|
|[31.6253601348306...|
|[31.6548096756927...|
|[31.8093003166791...|
|[31.8164283341993...|
|[31.8186165667690...|
|[31.8293464559211...|
|[31.8530748017465...|
|[31.8648325480987...|
|[31.9048571310136...|
|[31.9365486184489...|
|[31.9453957483445...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = lr_model.transform(unlabeled_data)

In [41]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.7377203726281...|449.85627807858054|
|[30.8162006488763...|282.20818683546327|
|[31.2834474760581...| 568.2729963524341|
|[31.3123495994443...| 443.7941016085322|
|[31.5257524169682...|448.22938457116925|
|[31.5316044825729...|432.89180664703053|
|[31.5702008293202...| 562.7778028198202|
|[31.5761319713222...| 542.2079126324022|
|[31.6098395733896...|426.81576727610764|
|[31.6253601348306...| 380.8337393752495|
|[31.6548096756927...|468.28266891103453|
|[31.8093003166791...| 546.5447924946088|
|[31.8164283341993...| 518.8223808333264|
|[31.8186165667690...|447.25752748274135|
|[31.8293464559211...|383.25034120065084|
|[31.8530748017465...|461.00645321559955|
|[31.8648325480987...|450.15917739312795|
|[31.9048571310136...|490.35141951175433|
|[31.9365486184489...|  440.243639364689|
|[31.9453957483445...| 661.6587092156005|
+--------------------+------------